<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/ChatHaruhi%E7%9A%84%E5%B0%8F%E8%AF%B4%E6%95%B0%E6%8D%AE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 统计换行数量
- [x] 统计引号数量
- [ ] prompt改写成多样化的prompt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Wuxia/input/all_direct

part_0.txt   part_1.txt  part_3.txt  part_5.txt  part_7.txt  part_9.txt
part_10.txt  part_2.txt  part_4.txt  part_6.txt  part_8.txt


In [ ]:
fname = "/content/drive/MyDrive/Wuxia/input/all_direct/part_10.txt"

import json

datas = []

with open(fname, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip() == "":
            continue
        data = json.loads(line)
        datas.append(data)

In [ ]:
print(len(datas))
print(datas[0].keys())

20205
dict_keys(['source', 'text'])


In [ ]:
def count_quotes(text):

    quote_chars = '''「」"“”'''
    count = 0
    for char in text:
        if char in quote_chars:
            count += 1
    return count

In [ ]:
def count_paragraphs(text):
    lines = text.split("\n")
    count = 0
    for line in lines:
        if line.strip() != "":
            count += 1
    return count

In [ ]:
from tqdm import tqdm
n_paragraphs = []
n_quotes = []

for data in tqdm(datas):
    n_paragraphs.append(count_paragraphs(data["text"]))
    n_quotes.append(count_quotes(data["text"]))

100%|██████████| 20205/20205 [00:06<00:00, 3006.08it/s]


对于一个字符串line，检查里面的'“'或者'"'或者'”'出现位置。并把所有位置合并为一个pos的list，如果pos的长度为偶数，则把奇数位置替换为「把偶数位置替换为」，请用python为我实现

In [ ]:
def change_quotes(text):
    lines = text.split("\n")

    quotes = ['“', '”', '"']

    new_text = ""
    for line in lines:
        if line.strip() == "":
            continue
        positions = []
        for i, char in enumerate(line):
            if char in quotes:
                positions.append(i)
        if len(positions) % 2 == 0:
            for i in range(len(positions)):
                if i % 2 != 0:
                    line = line[:positions[i]] + '」' + line[positions[i]+1:]
                else:
                    line = line[:positions[i]] + '「' + line[positions[i]+1:]
        new_text += line + "\n"
    return new_text


In [ ]:
prompt_templates = """Proceed with the paragraph, maintaining the same style of language.
Keep writing the paragraph, using a similar linguistic style.
Continue the paragraph, ensuring the language style remains consistent.
Go on with the paragraph in a similar style of language.
Carry on writing the paragraph, sticking to the current language style.
Extend the paragraph, keeping the language tone and style similar.
Resume the paragraph, mirroring the existing language style.
Continue crafting the paragraph, maintaining the established language style.
Keep extending the paragraph, following the same style of language.
Proceed with extending the paragraph, preserving the linguistic style.
Please compose a paragraph maintaining a uniform style throughout.
Write a paragraph ensuring that the style is consistent from start to finish.
Your assignment is to craft a paragraph with a consistent stylistic approach.
Create a paragraph where the style remains constant throughout.
Construct a paragraph, keeping the writing style uniform in the entire passage.
Develop a paragraph that demonstrates consistency in style.
Focus on writing a paragraph with a steady and unvarying style.
Your challenge is to author a paragraph that maintains a consistent style.
Compose a paragraph, ensuring stylistic consistency throughout.
Produce a paragraph in which the style does not vary from beginning to end.
Your task is to write paragraph in a consistent style
对于下面的Paragraph，使用一致性的风格进行续写
请继续下面的段落，并保持原有的写作风格一致。
在接下来的写作中，延续这一段落的风格，并进行扩展。
针对以下段落，以相同的风格进行补充写作。
续写下面的段落，并确保风格与原文保持一致。
请以与下面段落相同的风格，进行创作性的续写。
保持原段落的写作风格，对其进行扩展和深化。
延续以下段落的风格，进行相应的续写工作。
请在下面的段落后续写，确保风格的连贯性。
继续这一段落的故事，注意保持原有风格的一致性。
针对下文，以一种风格上连贯的方式进行创作性扩写。"""

prompt_templates = prompt_templates.split("\n")

for i in range(len(prompt_templates)):
    prompt_templates[i] = prompt_templates[i].strip()

In [ ]:
!pip install -q tiktoken
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
#@title 定义strong_divide
def divide_str(s, sep=['\n', '.', '。'], consider_quote = True ):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len + mid_len - 2)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            if consider_quote == True:
                if s[sep_pos + 1] in ['"', "'","“","」"]:
                    sep_pos += 1

            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s):
    # left, right = divide_str(s)

    # if right != '':
    #     return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

def super_divide( s, max_len_token = 300, iter_deep = 1):
    n_token = len(enc.encode(s))
    if n_token <= max_len_token or iter_deep > 2:
        return [s]

    left, right = strong_divide(s)
    return super_divide(left, max_len_token, iter_deep+1) + \
            super_divide(right, max_len_token, iter_deep+1)

In [ ]:
def segment_paragraph(text):
    raw_lines = text.split("\n")
    lines = []

    all_token_count = 0

    for line in raw_lines:
        line = line.strip()
        if line == "":
            continue
        n_token = len(enc.encode(line))
        all_token_count += n_token

        if n_token > 600:
            segments = super_divide(line)
            lines.extend(segments)
        else:
            lines.append(line)

    seg_token = all_token_count // 3

    if seg_token > 1500:
        seg_token = 1500

    current_seg_token = 0
    current_seg = ""

    segments = []
    for line in lines:
        line = line.strip('"#-。，, \n')
        if line.strip() == "":
            continue
        n_token = len(enc.encode(line))
        if current_seg_token + n_token > seg_token:
            if current_seg_token > 0:
                segments.append(current_seg.strip())
                current_seg = line
                current_seg_token = n_token
        else:
            current_seg += "\n" + line
            current_seg_token += n_token

    if current_seg_token > 0:
        segments.append(current_seg.strip())

    return segments

In [ ]:
print(10//3)

3


In [ ]:
source_pool = set()

In [ ]:
def find_first_sep( text ):
    first_sep = 9999
    first_sep_second_choice = 9999

    find_first_ket = text[:50].find("」",0)

    if find_first_ket > 0:
        return find_first_ket

    if len(text) <= 50:
        return len(text)
    for i, c in enumerate(text):
        if i > 50:
            break
        if c in ['。','\n']:
            first_sep = min(first_sep, i)
        elif c in ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']:
            first_sep_second_choice = min(first_sep_second_choice, i)

    for i in range(5):
        if len(text) > first_sep and text[first_sep+1] in ['\n', '.', '，', '、', ';', ',', '；',\
                    '：', '！', '？', '(', ')', '”', '“', \
                    '’', '‘', '[', ']', '{', '}', '<', '>', \
                    '/', '''\''', '|', '-', '=', '+', '*', '%', \
                '$', '''#''', '@', '&', '^', '_', '`', '~',\
                    '·', '…','「','」']:
            first_sep += 1

    if first_sep <= 50:
        return first_sep
    elif first_sep_second_choice <= 50:
        return first_sep_second_choice
    else:
        return 50

In [ ]:
count = 0

import random

save_datas = []

for i in tqdm(range(len(datas))):
    if n_quotes[i] < 8 or n_paragraphs[i] < 9:
        continue

    source = datas[i]["source"]
    source_pool.add(source)

    text = datas[i]["text"]
    text = change_quotes( text )
    segments = segment_paragraph( text )

    context = random.choice(prompt_templates)

    context += "\nParagraph:\n###\n" + segments[0] + "\n###\n"

    first_sep = find_first_sep(segments[1])

    context += segments[1][:first_sep+1]

    target = segments[1][first_sep+1:]

    for j in range(2, len(segments)):
        target += "\n\n" + segments[j]

    save_data = {
        "context": context,
        "target": target
    }

    save_datas.append(save_data)


100%|██████████| 20205/20205 [01:41<00:00, 199.99it/s]


In [ ]:
import json

save_name = "/content/classic_novel_writing_18k.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data in save_datas:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
print(context)

Your challenge is to author a paragraph that maintains a consistent style.
Paragraph:
###
八月末的一天清早，白嘉轩起来洗脸漱口时，他的冒死破禁而且显出怀孕徵兆的妻子仙草正坐在纺线车前嗡嗡嗡嗡地转动着车把儿，锭子上已经结下一枚茭白大小的白色线穗了。母亲也早已起来，在自个独居的里屋炕上摇转着纺车。他坐在父亲在世时常坐的那把靠背椅子上，喝看酽茶，用父亲死後留下的那把白铜水烟袋过著早瘾。父亲死後，他每天晚上在母亲落枕前和清早起床後都到里屋里坐一会儿。两架纺车嗡嗡吱吱的声音互相衔接，互相重合，此声间歇，彼声响起，把沉稳和谐的气氛弥漫到四合院的每一个角落。白嘉轩沉浸在这古老悠远而又新鲜活泼的乐曲里，浑身的筋骨和血液就鼓涨起来
长工鹿三把犁铧套绳收拾齐备，从马号里牵出红马拴在院子里的石雕拴马桩上，扯着大步走进院庭，大声询问种子的事。嘉轩从里屋走出来：「你先喝口茶。」鹿三站在院庭里说他不喝，仍然询问麦子和豌豆掺和的比例，二八还是三七？嘉轩说：「这块地种药材。种子你甭管，我拿着。」说着喷出一口烟，吹净水烟筒里的烟灰，放下水烟壶，喝下最後一盅茶，就赳赳地走出街门，进入马号。鹿三解下红马牵着，套上犁杖。嘉轩扛起沉重的铁齿大耙子，腋下挟着一把镢头和一把竹条扫帚。鹿三回过头问：「你拿扫帚做啥？」嘉轩也不解释：「拿就是有用嘛。」鹿三就不再问。主仆二人走过街巷，出了村子，走下河滩，红马拖着空犁在田间土路上撞出瞠瞠瞠的声响
田野已经改换过另一种姿容，斑斓驳杂的秋天的色彩像羽毛一样脱光褪尽荡然无存了，河川里呈现出一种喧闹之後的沉静。灌渠渠沿和井台上堆积着刚刚从田地里清除出来的包谷秆子。麦子播种几近尾声，刚刚播种不久的田块裸露着湿漉漉的泥土，早种的田地已经泛出麦苗幼叶的嫩绿。秋天的淫雨季节已告结束，长久弥漫在河川和村庄上空的阴霾和沉闷已全部廓清。大地简洁而素雅，天空开阔而深远。清晨的冷气使人精神抖擞
###




# 额外20k数据的获取

下面从全小说库进行探查，但是这个时候我们只考虑对话较多的数据

In [ ]:
novel2count = {}

import json

datas = []
for part_id in tqdm(range(0, 10)):
    fname = "/content/drive/MyDrive/Wuxia/input/all_direct/part_"+ str(part_id) +".txt"


    with open(fname, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip() == "":
                continue
            data = json.loads(line)
            source = data["source"]
            if source in source_pool:
                continue

            if source not in novel2count:
                novel2count[source] = 0

            if novel2count[source] > 20:
                continue

            n_quote = count_quotes(data["text"])
            if n_quote >= 110:
                continue

            novel2count[source] += 1
            datas.append(data)

    print(len(datas))

 10%|█         | 1/10 [00:06<00:58,  6.46s/it]

4949


 20%|██        | 2/10 [00:17<01:11,  8.97s/it]

10146


 30%|███       | 3/10 [00:23<00:55,  7.95s/it]

...

40409


 90%|█████████ | 9/10 [01:43<00:14, 14.52s/it]

45464


100%|██████████| 10/10 [01:58<00:00, 11.81s/it]

50427


In [ ]:
count = 0

import random

save_datas = []

for i in tqdm(range(len(datas))):
    source = datas[i]["source"]
    source_pool.add(source)

    text = datas[i]["text"]
    text = change_quotes( text )
    segments = segment_paragraph( text )

    if len(segments) < 3:
        continue

    context = random.choice(prompt_templates)

    context += "\nParagraph:\n###\n" + segments[0] + "\n###\n"

    first_sep = find_first_sep(segments[1])

    context += segments[1][:first_sep+1]

    target = segments[1][first_sep+1:]

    for j in range(2, len(segments)):
        target += "\n\n" + segments[j]

    save_data = {
        "context": context,
        "target": target
    }

    save_datas.append(save_data)


100%|██████████| 50427/50427 [04:56<00:00, 169.95it/s]


In [ ]:
import json

save_name = "/content/drive/MyDrive/Wuxia/input/all_direct/dialogueful_novel.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data in save_datas:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
print(len(datas))

87620


In [ ]:
for data in tqdm(datas):
    n_quotes.append(count_quotes(data["text"]))

100%|██████████| 98666/98666 [00:29<00:00, 3312.06it/s]


In [ ]:
n_quotes_sorted = sorted(n_quotes, reverse=True)

In [ ]:
print(n_quotes_sorted[2000])

116


In [ ]:
print(n_quotes_sorted[2000])

132
